In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

%load_ext nvcc_plugin

In [64]:
%%cu
#include <stdio.h> //Obliczenie wektorów na CPU
#define N (512*512)
#define M (10000)
  int main()
  {
     float data[N]; int count = 0;
     for(int i = 0; i < N; i++)
     {
        data[i] = 1.0f * i / N;
        for(int j = 0; j < M; j++)
        {
           data[i] = data[i] * data[i] - 0.25f;
        }
     }
     int sel = 256;
     printf("data[%d] = %f\n", sel, data[sel]);
  }


data[256] = -0.207107



In [59]:
%%cu
#include<stdio.h>
#define N (512*512) //Obliczenie wektorów na GPU
#define M (10000)

  __global__ void cudakernel(float *buf)
  {
     int i = threadIdx.x + blockIdx.x * blockDim.x;
     buf[i] = 1.0f * i / N;
     for(int j = 0; j < M; j++)
        buf[i] = buf[i] * buf[i] - 0.25f;
  }

  int main()
  {
     float data[N]; int count = 0;
     float *d_data;
     cudaMalloc(&d_data, N * sizeof(float));
     cudakernel<<<N/256, 256>>>(d_data);
     cudaMemcpy(data, d_data, N * sizeof(float), cudaMemcpyDeviceToHost);
     cudaFree(d_data); 

     int sel;
     scanf("%d", &sel);
     printf("data[%d] = %f\n", sel, data[sel]);
  }


data[256] = -0.207107



In [77]:
%%cu
#include <stdlib.h>
#include <stdio.h>
bool isJulia( float x, float y, float maxX_2, float maxY_2 )
    {
        float z_r = 0.8 * (float) (maxX_2 - x) / maxX_2;
        float z_i = 0.8 * (float) (maxY_2 - y) / maxY_2;

        float c_r = -0.8;
        float c_i = 0.156;
        for( int i=1 ; i<100 ; i++ )
        {
        float tmp_r = z_r*z_r - z_i*z_i + c_r;
        float tmp_i = 2*z_r*z_i + c_i;

        z_r = tmp_r;
        z_i = tmp_i;

        if( sqrt( z_r*z_r + z_i*z_i ) > 1000 )
            return false;
        }
        return true;
    }


  

    int main(void)
    {
      const int dimx = 768, dimy = 768;
      int i, j;

      unsigned char * data = new unsigned char[dimx*dimy*3];

      for( int z=0 ; z<512 ; z++ ) // ustawienie ilości obliczeń punktów fraktalu Julii
      {
            for (j = 0; j < dimy; ++j)
            {
              for (i = 0; i < dimx; ++i)
              {
                if( isJulia(i,j,dimx/2,dimy/2) == true )
                {
                data[3*j*dimx + 3*i + 0] = (unsigned char)255;  /* red */
                data[3*j*dimx + 3*i + 1] = (unsigned char)0;  /* green */
                data[3*j*dimx + 3*i + 2] = (unsigned char)0;  /* blue */
                }
                else
                {
                data[3*j*dimx + 3*i + 0] = (unsigned char)255;  /* red */
                data[3*j*dimx + 3*i + 1] = (unsigned char)255;  /* green */
                data[3*j*dimx + 3*i + 2] = (unsigned char)255;  /* blue */
                }
              }
            }
      }

      //writePPMImage( data, dimx, dimy, 3, "out_cpu.ppm" ); //tworzenie pliku ppm z fraktalem Julii 
      delete [] data;


      return 0;
    }

In [78]:
    %%cu
    #include <stdio.h>

    __device__ bool isJulia( float x, float y, float maxX_2, float maxY_2 )
    {
        float z_r = 0.8 * (float) (maxX_2 - x) / maxX_2;
        float z_i = 0.8 * (float) (maxY_2 - y) / maxY_2;

        float c_r = -0.8;
        float c_i = 0.156;
        for( int i=1 ; i<100 ; i++ )
        {
        float tmp_r = z_r*z_r - z_i*z_i + c_r;
        float tmp_i = 2*z_r*z_i + c_i;

        z_r = tmp_r;
        z_i = tmp_i;

        if( sqrt( z_r*z_r + z_i*z_i ) > 1000 )
            return false;
        }
        return true;
    }

    __global__ void kernel( unsigned char * im, int dimx, int dimy )
    {
        //int tid = blockIdx.y*gridDim.x + blockIdx.x;
        int tid = blockIdx.x*blockDim.x + threadIdx.x;
        tid *= 3;
        if( isJulia((float)blockIdx.x, (float)threadIdx.x, (float)dimx/2, (float)dimy/2)==true )
        {
        im[tid] = 255;
        im[tid+1] = 0;
        im[tid+2] = 0;
        }
        else
        {
        im[tid] = 255;
        im[tid+1] = 255;
        im[tid+2] = 255;
        }

    }

    int main()
    {
        int dimx=768, dimy=768;

        //on cpu
        unsigned char * im = (unsigned char*) malloc( 3*dimx*dimy );

        //on GPU
        unsigned char * im_dev;

        //allocate mem on GPU
        cudaMalloc( (void**)&im_dev, 3*dimx*dimy ); 

        //launch kernel. 
        for( int z=0 ; z<512 ; z++ ) // ustawienie ilości obliczeń punktów fraktalu Julii
        {
                kernel<<<dimx,dimy>>>(im_dev, dimx, dimy);
        }

                cudaMemcpy( im, im_dev, 3*dimx*dimy, cudaMemcpyDeviceToHost );

                //writePPMImage( im, dimx, dimy, 3, "out_gpu.ppm" ); //tworzenie pliku ppm z fraktalem Julii 

                free( im );
                cudaFree( im_dev );
    }